# Business Questions Analysis

This notebook explores which business questions can be answered with the current database, which cannot, and proposes new metrics for future analysis.

---

## Table of Contents

1. [Possible Business Questions](#possible-business-questions)
    - [1. Which day of the week has the most sales?](#day-of-week-sales)
    - [2. What is the average amount paid by payment method?](#average-amount-by-payment-method)
2. [Impossible Business Questions](#impossible-business-questions)
    - [1. How many orders were paid using more than one payment method?](#multi-payment-orders)
    - [2. How many users register per month?](#monthly-user-registrations)
    - [3. How many products are currently out of stock?](#out-of-stock-products)
3. [New Business Questions](#new-business-questions)


In [1]:
from database.database_connection import get_session
from database.models import (
    Usuario, Categoria, Producto, Orden, DetalleOrden,
    DireccionEnvio, Carrito, MetodoPago, OrdenMetodoPago,
    ResenaProducto, HistorialPago
)
from sqlalchemy import select, func, extract

In [2]:
session = get_session()

## 1. Possible Business Questions <a name="possible-business-questions"></a>

### 1.1 Which day of the week has the most sales? <a name="day-of-week-sales"></a>

We analyze the sales distribution by day of the week using the `fecha_orden` field from the `Orden` table.

---

In [3]:
stmt_check = (
    select(
        Orden.fecha_orden,
        extract('dow', Orden.fecha_orden).label('dow')
    )
    .limit(10)
)
for fecha, dow in session.execute(stmt_check):
    print(fecha, dow)

2024-09-09 09:10:27 1
2024-09-29 09:10:27 0
2024-11-03 09:10:27 0
2024-09-14 09:10:27 6
2025-04-11 09:10:27 5
2025-06-26 09:10:27 4
2024-08-14 09:10:27 3
2025-04-17 09:10:27 4
2024-08-03 09:10:27 6
2024-08-26 09:10:27 1


### 1) ¿Qué día de la semana se generan más ventas?



In [4]:
from sqlalchemy import select, func, extract
from database.models import Orden

def imprimir_ventas_por_dia_semana(session):
    stmt = (
        select(
            extract('dow', Orden.fecha_orden).label('dia_semana'),
            func.count().label('cantidad')
        )
        .group_by('dia_semana')
        .order_by(func.count().desc())
    )

    result = session.execute(stmt).all()

    dias_nombre = {
        0: 'Domingo',
        1: 'Lunes',
        2: 'Martes',
        3: 'Miércoles',
        4: 'Jueves',
        5: 'Viernes',
        6: 'Sábado'
    }

    if not result:
        print("No hay datos de ventas.")
        return

    # Día con más ventas
    dia_top, cantidad_top = result[0]
    print(f"El día con más ventas es: {dias_nombre.get(int(dia_top), 'Desconocido')} con {cantidad_top} ventas\n")

    # Lista los demás días con sus cantidades
    print("Siguientes días con ventas:")
    for dia, cantidad in result[1:]:
        print(f"{dias_nombre.get(int(dia), 'Desconocido')}: {cantidad} ventas")

imprimir_ventas_por_dia_semana(session)


El día con más ventas es: Domingo con 1517 ventas

Siguientes días con ventas:
Sábado: 1486 ventas
Lunes: 1415 ventas
Miércoles: 1412 ventas
Viernes: 1407 ventas
Martes: 1385 ventas
Jueves: 1378 ventas


---
## 2. Impossible Business Questions <a name="impossible-business-questions"></a>

### 2.1 How many orders were paid using more than one payment method? <a name="multi-payment-orders"></a>

Currently, the `OrdenMetodoPago` table does not allow tracking multiple payment methods per order, as each order appears only once.  
To enable this analysis, consider implementing a Slowly Changing Dimension (SCD) Type 2 for payment methods.



### 2.2 How many users register per month? <a name="monthly-user-registrations"></a>

This cannot be answered because the `Usuario` table does not contain a registration date.  
Adding a `created_at` column would allow monthly registration analysis.


### 2.3 How many products are currently out of stock? <a name="out-of-stock-products"></a>

The code below shows that there are no products out of stock in the current database snapshot.  
This could mean that out-of-stock products are deleted or that there were none at the time of data extraction.


In [5]:
stmt = select(Producto).where(Producto.stock == 0)

result = session.execute(stmt).scalars().all()
print(f"Cantidad de productos fuera de stock: {len(result)}")


Cantidad de productos fuera de stock: 0


## 3. New Business Questions <a name="new-business-questions"></a>


Cuantos productos tiene un carrito promedio. Por como se entrego la data, no es posible saber si los usuarios tienen siempre un mismo carrito y que despues hacen checkout, o si lo que agregan se termina sacando o comprando. 
Cambiaria que a un usuariose le asigne un carrito cuando agrega un producto, y se mantiene el mismo carrito hasta que se realice una compra. Si se vacia el carrito se sigue manteniendo el mismo pero esta vacio. Asi se podria a lo largo comparar como fueron evolucionando los carritos de un usuario.